<a href="https://colab.research.google.com/github/ioahKwon/Machine-Learning-Deep-Learning/blob/master/%5B%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D%EA%B3%BC%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_Week_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[머신러닝과딥러닝] 수업 정리 Week #12**

### _2020.08._ SKKU Lecture Summary by IOAH


---





### 1. Convolutional Neural Networks를 이용한 MNIST DATASET 분류

이번에는 CNN을 이용하여 MNIST 데이터를 분류하고 CNN의 구조, 파라미터 등을 바꿔가며 성능의 변화를 확인해보도록 하자. 실습을 위해 필요한 module을 import해주고 미리 hyper-parameter를 선언해둔다. 그 후 데이터를 로드하고 이를 trainset과 test set으로 분할시킨다. 이후 데이터 전처리 과정을 거친다. 위 과정은 아래와 같다.

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [ ]:
batch_size = 128
num_classes = 10
epoch = 12

In [ ]:
img_rows, img_cols = 28,28
(x_train,y_train),(x_test,y_test) = mnist.load_data()
if K.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0],1,img_rows,img_cols)
  x_test = x_test.reshape(x_test.shape[0],1,img_rows,img_cols)
  input_shape = (1,img_rows,img_cols)
else: 
  x_train = x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
  x_test = x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
  input_shape = (img_rows,img_cols,1)

In [ ]:
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255

y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)

그 후 model을 sequential()을 이용해 쌓는다. Conv2D를 통해 CNN Layer를 쌓고 maxpooling layer를 통해 pooling layer를 쌓는다. 그 후 0.25로 dropout layer를 overfitting을 막기 위해 쌓으며 이를 flatten함수를 통해 평탄화시키고 Fully connected layer로 activation function은 relu로, dropout 비율은 0.5, output layer의 activation function은 softmax로 쌓아서 모델을 완성한다.

또한 모델에 사용될 loss function은 categorical_crossentropy로 주고 optimizer는 Adadelta로 주기로 하자. 평가지표는 accuracy로 정한다. 

이후 모델은 train set에 대해 학습시킨다.


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,
          batch_size=batch_size,
          epochs = epoch,
          verbose=1)

Epoch 1/12
60000/60000 [==============================] - 2s 40us/step - loss: 0.3408 - accuracy: 0.8971
Epoch 2/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.1399 - accuracy: 0.9584
Epoch 3/12
60000/60000 [==============================] - 2s 36us/step - loss: 0.1049 - accuracy: 0.9689
Epoch 4/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0900 - accuracy: 0.9726
Epoch 5/12
60000/60000 [==============================] - 2s 36us/step - loss: 0.0794 - accuracy: 0.9762
Epoch 6/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0708 - accuracy: 0.9781
Epoch 7/12
60000/60000 [==============================] - 2s 36us/step - loss: 0.0639 - accuracy: 0.9803
Epoch 8/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0601 - accuracy: 0.9818
Epoch 9/12
60000/60000 [==============================] - 2s 37us/step - loss: 0.0548 - accuracy: 0.9833
Epoch 10/12
60000/60000 [==============================

In [ ]:
score=model.evaluate(x_test,y_test,verbose=0)
print('Test loss : ',score[0])
print('Test acc : ',score[1])

Test loss :  0.0369082065519935
Test acc :  0.9873999953269958


결과가 꽤 잘 나온 것을 확인할 수 있다. 저번 11주차 실습에 비해서 꽤 성능이 향상됐음을 볼 수 있다.

CNN 모델의 성능을 향상시키기 위해선 무엇이 또 필요할까? 우선 CNN LAYER를 한층 더 쌓아보자.

In [ ]:
model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape))
model2.add(Conv2D(32,(3,3),activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Dropout(0.25))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(num_classes, activation='softmax'))

In [ ]:
model2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model2.fit(x_train,y_train,
          batch_size=batch_size,
          epochs = epoch,
          verbose=1)
score=model2.evaluate(x_test,y_test,verbose=0)
print('Test loss : ',score[0])
print('Test acc : ',score[1])

Epoch 1/12
60000/60000 [==============================] - 4s 60us/step - loss: 0.2990 - accuracy: 0.9079
Epoch 2/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.1018 - accuracy: 0.9696
Epoch 3/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.0744 - accuracy: 0.9779
Epoch 4/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.0633 - accuracy: 0.9805
Epoch 5/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.0574 - accuracy: 0.9831
Epoch 6/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.0478 - accuracy: 0.9855
Epoch 7/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.0461 - accuracy: 0.9866
Epoch 8/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.0426 - accuracy: 0.9871
Epoch 9/12
60000/60000 [==============================] - 3s 55us/step - loss: 0.0394 - accuracy: 0.9882
Epoch 10/12
60000/60000 [==============================

그러면 Train set에 대해선 무려 98.9%, test set에 대해선 99.1%의 경이로운 정확도를 보인다. 

이외에도 Filter size등 다른 매개변수를 조금만 조정해주면 test set의 accuracy가 많이 올라가는 것을 확인할 수 있다.

따라서 적절하게 매개변수를 조절해주거나 또는 CNN Layer를 더 deep 하게 쌓아주면 성능이 올라감을 확인할 수 있다.
